# 4 Advanced Modeling - Specialized Packages

In this notebook we will use more advanced models on our two wells to see if we can further improve the MAPE scores compared to our base models. We will be using advanced algorithnms, packages and framemworks such as pmdraima,KATS,GREYkite. 

## TABLE OF CONTENTS:
* [4.1 Import Relevant Package](#1)
* [4.2 Import CSV](#2)
* [4.3 Examine Data](#3)
* [4.4 Preprocessing and training for well with a typical decline curve profile](#4)
    * [4.4.1 Auto-ARIMA](#5)
    * [4.4.2 KATS](#6)
        * [4.4.2.1 KATS - ARIMA](#7)
        * [4.4.2.2 KATS - fbProphet](#8)
        * [4.4.2.3 KATS - Theta model](#9)
        * [4.4.2.4 KATS - Harmonic Regression model](#10)
        * [4.4.2.5 KATS - LSTM model](#11)
        * [4.4.2.6 KATS - Ensemble model](#12)
    * [4.4.3 SKTIME](#13)
        * [4.4.3.1 SKTIME-Theta model](#14)
        * [4.4.3.2 SKTIME-TBATS](#15)
        * [4.4.3.3 SKTIME-Polynomial Trend](#16)
    * [4.4.4 Silverkite](#17)
    * [4.5 Performance of Models](#18)
        
            
        
            
            
            
            
            
            
        
            
            
            
            
        




 <b>4.1 Import Relevant Package <b/> <a class="anchor" id="1"></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pmdarima.arima import auto_arima
from kats.consts import TimeSeriesData
from kats.models.sarima import SARIMAModel, SARIMAParams
from kats.models.prophet import ProphetModel, ProphetParams
from kats.models.theta import ThetaModel, ThetaParams
from kats.models.quadratic_model import QuadraticModel, QuadraticModelParams
from kats.models.lstm import LSTMModel, LSTMParams
import warnings as warnings
from kats.models.ensemble.ensemble import EnsembleParams, BaseModelParams
from kats.models.ensemble.kats_ensemble import KatsEnsemble
from kats.models import ( arima, holtwinters,linear_model,prophet, quadratic_model,sarima,theta)
from sktime.forecasting.theta import ThetaForecaster
from sktime.forecasting.tbats import TBATS
from sktime.forecasting.trend import PolynomialTrendForecaster
import plotly
from greykite.algo.changepoint.adalasso.changepoint_detector import ChangepointDetector
from greykite.algo.forecast.silverkite.constants.silverkite_holiday import SilverkiteHoliday
from greykite.algo.forecast.silverkite.constants.silverkite_seasonality import SilverkiteSeasonalityEnum
from greykite.algo.forecast.silverkite.forecast_simple_silverkite_helper import cols_interact
from greykite.common import constants as cst
from greykite.common.features.timeseries_features import build_time_features_df
from greykite.common.features.timeseries_features import convert_date_to_continuous_time
from greykite.framework.benchmark.data_loader_ts import DataLoaderTS
from greykite.framework.templates.autogen.forecast_config import EvaluationPeriodParam
from greykite.framework.templates.autogen.forecast_config import ForecastConfig
from greykite.framework.templates.autogen.forecast_config import MetadataParam
from greykite.framework.templates.autogen.forecast_config import ModelComponentsParam
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results
warnings.filterwarnings("ignore",category=UserWarning)

C:\Users\ferna\anaconda3\envs\timeseries\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 <b>4.2 Import CSV <b/> <a class="anchor" id="2"></a>

In [2]:
df = pd.read_csv('Monthly.csv')
df1Performance = pd.read_csv('df1Performance.csv')
df2Performance = pd.read_csv('df2Performance.csv')

 <b>4.3 Examine Data <b/>  <a class="anchor" id="3"></a>

Drop index and level_0 columns

In [3]:
df.drop(columns=['index','level_0'], inplace=True)

Call head of data.

In [4]:
df.head(2)

,DATEPRD,AVG_DOWNHOLE_PRESSURE,AVG_DOWNHOLE_TEMPERATURE,AVG_DP_TUBING,AVG_ANNULUS_PRESS,AVG_CHOKE_SIZE_P,AVG_WHP_P,AVG_WHT_P,DP_CHOKE_SIZE,BORE_OIL_VOL,...,AVG_DP_TUBING.shifted3,AVG_ANNULUS_PRESS.shifted3,AVG_CHOKE_SIZE_P.shifted3,AVG_WHP_P.shifted3,AVG_WHT_P.shifted3,DP_CHOKE_SIZE.shifted3,BORE_OIL_VOL.shifted3,BORE_GAS_VOL.shifted3,BORE_WAT_VOL.shifted3,BORE_WI_VOL.shifted3
0,2014-07-01,215.305839,108.150677,166.458710,0.0,47.269826,48.847129,56.742968,20.381968,15084.0,...,211.228320,0.0,22.489461,47.242292,27.762750,34.311375,631.0,0.0,0.0,0.0
1,2014-08-01,230.622097,105.879581,188.180742,0.0,29.241656,42.441258,41.679613,24.964645,6968.0,...,155.976129,0.0,46.416057,67.994387,55.577839,39.485194,20569.0,0.0,783.0,0.0


Make 'DATEPRD' as index

In [5]:
df.set_index('DATEPRD',inplace=True)

Call head of data.

In [6]:
df.head(2)

,AVG_DOWNHOLE_PRESSURE,AVG_DOWNHOLE_TEMPERATURE,AVG_DP_TUBING,AVG_ANNULUS_PRESS,AVG_CHOKE_SIZE_P,AVG_WHP_P,AVG_WHT_P,DP_CHOKE_SIZE,BORE_OIL_VOL,BORE_GAS_VOL,...,AVG_DP_TUBING.shifted3,AVG_ANNULUS_PRESS.shifted3,AVG_CHOKE_SIZE_P.shifted3,AVG_WHP_P.shifted3,AVG_WHT_P.shifted3,DP_CHOKE_SIZE.shifted3,BORE_OIL_VOL.shifted3,BORE_GAS_VOL.shifted3,BORE_WAT_VOL.shifted3,BORE_WI_VOL.shifted3
DATEPRD,,,,,,,,,,,,,,,,,,,,,
2014-07-01,215.305839,108.150677,166.458710,0.0,47.269826,48.847129,56.742968,20.381968,15084.0,0.0,...,211.228320,0.0,22.489461,47.242292,27.762750,34.311375,631.0,0.0,0.0,0.0
2014-08-01,230.622097,105.879581,188.180742,0.0,29.241656,42.441258,41.679613,24.964645,6968.0,0.0,...,155.976129,0.0,46.416057,67.994387,55.577839,39.485194,20569.0,0.0,783.0,0.0


Convert index to datetime

In [7]:
df.index = pd.to_datetime(df.index)

Call shape of dataframe

In [8]:
df.shape

(285, 25)

Call column names

In [9]:
df.columns

Index(['AVG_DOWNHOLE_PRESSURE', 'AVG_DOWNHOLE_TEMPERATURE', 'AVG_DP_TUBING',
       'AVG_ANNULUS_PRESS', 'AVG_CHOKE_SIZE_P', 'AVG_WHP_P', 'AVG_WHT_P',
       'DP_CHOKE_SIZE', 'BORE_OIL_VOL', 'BORE_GAS_VOL', 'BORE_WAT_VOL',
       'BORE_WI_VOL', 'NPD_WELL_BORE_NAME', 'AVG_DOWNHOLE_PRESSURE.shifted3',
       'AVG_DOWNHOLE_TEMPERATURE.shifted3', 'AVG_DP_TUBING.shifted3',
       'AVG_ANNULUS_PRESS.shifted3', 'AVG_CHOKE_SIZE_P.shifted3',
       'AVG_WHP_P.shifted3', 'AVG_WHT_P.shifted3', 'DP_CHOKE_SIZE.shifted3',
       'BORE_OIL_VOL.shifted3', 'BORE_GAS_VOL.shifted3',
       'BORE_WAT_VOL.shifted3', 'BORE_WI_VOL.shifted3'],
      dtype='object')

Create a list of all columns to remove

In [10]:
list_to_remove = ['AVG_DOWNHOLE_PRESSURE.shifted3',
 'AVG_DOWNHOLE_TEMPERATURE.shifted3',
 'AVG_DP_TUBING.shifted3',
 'AVG_ANNULUS_PRESS.shifted3',
 'AVG_CHOKE_SIZE_P.shifted3',
 'AVG_WHP_P.shifted3',
 'AVG_WHT_P.shifted3',
 'DP_CHOKE_SIZE.shifted3',
 'BORE_OIL_VOL.shifted3',
 'BORE_GAS_VOL.shifted3',
 'BORE_WAT_VOL.shifted3',
 'BORE_WI_VOL.shifted3','BORE_WI_VOL']

Drop all unwanted columns.

In [11]:
df.drop(columns=list_to_remove,inplace=True)

Call head of dataframe.

In [12]:
df.head(2)

,AVG_DOWNHOLE_PRESSURE,AVG_DOWNHOLE_TEMPERATURE,AVG_DP_TUBING,AVG_ANNULUS_PRESS,AVG_CHOKE_SIZE_P,AVG_WHP_P,AVG_WHT_P,DP_CHOKE_SIZE,BORE_OIL_VOL,BORE_GAS_VOL,BORE_WAT_VOL,NPD_WELL_BORE_NAME
DATEPRD,,,,,,,,,,,,
2014-07-01,215.305839,108.150677,166.458710,0.0,47.269826,48.847129,56.742968,20.381968,15084.0,0.0,6244.0,15/9-F-1 C
2014-08-01,230.622097,105.879581,188.180742,0.0,29.241656,42.441258,41.679613,24.964645,6968.0,0.0,4530.0,15/9-F-1 C


Call tail of dataframe

In [13]:
df.tail()

,AVG_DOWNHOLE_PRESSURE,AVG_DOWNHOLE_TEMPERATURE,AVG_DP_TUBING,AVG_ANNULUS_PRESS,AVG_CHOKE_SIZE_P,AVG_WHP_P,AVG_WHT_P,DP_CHOKE_SIZE,BORE_OIL_VOL,BORE_GAS_VOL,BORE_WAT_VOL,NPD_WELL_BORE_NAME
DATEPRD,,,,,,,,,,,,
2016-03-01,218.160839,105.842548,184.348968,16.219032,12.305040,33.811903,37.630516,17.960387,3746.0,0.0,5035.0,15/9-F-15 D
2016-04-01,232.432300,105.733967,197.541600,16.938367,19.377015,34.890833,36.453000,11.502967,4545.0,29.0,5687.0,15/9-F-15 D
2016-05-01,209.717903,106.244226,184.899968,18.447258,16.015713,24.818000,42.118097,7.889194,4696.0,0.0,6007.0,15/9-F-15 D
2016-06-01,212.926967,105.923067,184.994100,16.132400,23.024772,27.932900,40.764933,14.086767,3466.0,0.0,5390.0,15/9-F-15 D
2016-07-01,308.101258,92.706710,299.597355,4.271226,7.057121,8.503903,12.570806,9.004000,828.0,0.0,1173.0,15/9-F-15 D


 <b>4.4 Preprocessing and training for well with a typical decline curve profile<b/> <a class="anchor" id="4"></a>

Create dataframe df1 for F-14 well.

In [14]:
df1 = df[df['NPD_WELL_BORE_NAME'] =='15/9-F-14'].copy()

Plot oil production rate

In [15]:
fig_dims = (10, 12)
fig, ax = plt.subplots(figsize=fig_dims)
ax.set_title('15/9-F-14',size=20)
sns.lineplot(x='DATEPRD', y='BORE_OIL_VOL', data=df1, ax=ax, color='black')

<AxesSubplot:title={'center':'15/9-F-14'}, xlabel='DATEPRD', ylabel='BORE_OIL_VOL'>

Remove irrelevant data

In [16]:
df1= df1[df1.index >= '2013-02-01'].copy()

Plot oil production rate

In [17]:
fig_dims = (10, 12)
fig, ax = plt.subplots(figsize=fig_dims)
ax.set_title('15/9-F-14',size=20)
sns.lineplot(x='DATEPRD', y='BORE_OIL_VOL', data=df1, ax=ax, color='black')

<AxesSubplot:title={'center':'15/9-F-14'}, xlabel='DATEPRD', ylabel='BORE_OIL_VOL'>

Examine dataframe shape.

In [18]:
df1.shape

(42, 12)

Call dataframe head.

In [19]:
df1.head()

,AVG_DOWNHOLE_PRESSURE,AVG_DOWNHOLE_TEMPERATURE,AVG_DP_TUBING,AVG_ANNULUS_PRESS,AVG_CHOKE_SIZE_P,AVG_WHP_P,AVG_WHT_P,DP_CHOKE_SIZE,BORE_OIL_VOL,BORE_GAS_VOL,BORE_WAT_VOL,NPD_WELL_BORE_NAME
DATEPRD,,,,,,,,,,,,
2013-02-01,242.421357,101.094214,209.964714,22.047036,96.248736,32.456643,87.408179,3.390214,25589.0,0.0,0.0,15/9-F-14
2013-03-01,245.233805,100.381593,208.418654,19.721192,61.694085,36.922413,81.552552,9.453654,18818.0,0.0,348.0,15/9-F-14
2013-04-01,244.286854,100.123739,206.884745,19.660531,60.121212,37.550047,82.163571,10.099634,8545.0,0.0,393.0,15/9-F-14
2013-05-01,238.605935,98.639355,206.072645,22.351516,97.713162,32.533387,87.742226,3.551355,27665.0,0.0,0.0,15/9-F-14
2013-06-01,244.610010,100.419686,211.495473,20.558532,85.809295,33.188373,87.409936,4.456317,22299.0,0.0,385.0,15/9-F-14


Call dataframe tail

In [20]:
df1.tail()

,AVG_DOWNHOLE_PRESSURE,AVG_DOWNHOLE_TEMPERATURE,AVG_DP_TUBING,AVG_ANNULUS_PRESS,AVG_CHOKE_SIZE_P,AVG_WHP_P,AVG_WHT_P,DP_CHOKE_SIZE,BORE_OIL_VOL,BORE_GAS_VOL,BORE_WAT_VOL,NPD_WELL_BORE_NAME
DATEPRD,,,,,,,,,,,,
2016-03-01,267.556452,99.625387,239.260194,21.033677,98.741093,28.296194,88.031548,1.885323,3755.0,0.0,0.0,15/9-F-14
2016-04-01,268.142267,99.594733,239.015767,22.148467,83.716599,29.126433,87.600967,3.120167,3114.0,0.0,0.0,15/9-F-14
2016-05-01,270.780226,99.769323,239.268000,22.514065,42.852574,31.512226,87.651548,5.693258,3214.0,0.0,0.0,15/9-F-14
2016-06-01,269.827700,100.053033,238.942100,23.435200,47.767466,30.885567,88.542800,5.303067,3073.0,0.0,0.0,15/9-F-14
2016-07-01,286.916323,98.268194,259.612484,12.011452,29.669248,27.303903,41.712613,7.113097,1326.0,0.0,0.0,15/9-F-14


 <b>4.4.1 Auto-ARIMA<b/> <a class="anchor" id="5"></a>

Create oil production rate time series

In [21]:
df1_Oil = df1['BORE_OIL_VOL']

Call time series tail

In [22]:
df1_Oil.tail()

DATEPRD
2016-03-01    3755.0
2016-04-01    3114.0
2016-05-01    3214.0
2016-06-01    3073.0
2016-07-01    1326.0
Name: BORE_OIL_VOL, dtype: float64

Split time series between Training and Testing

In [23]:
Train=df1_Oil.loc[:'2016-03-01']

In [24]:
Test=df1_Oil.loc['2016-04-01':]

In [25]:
Test

DATEPRD
2016-04-01    3114.0
2016-05-01    3214.0
2016-06-01    3073.0
2016-07-01    1326.0
Name: BORE_OIL_VOL, dtype: float64

Plot train set.

In [26]:
Train.plot() 

<AxesSubplot:title={'center':'15/9-F-14'}, xlabel='DATEPRD', ylabel='BORE_OIL_VOL'>

Plot test set.

In [27]:
Test.plot()

<AxesSubplot:title={'center':'15/9-F-14'}, xlabel='DATEPRD', ylabel='BORE_OIL_VOL'>

Create auto arima object. This model will auto-select the best model given our p , q , d space  and select the model with the lowest AIC score.

In [28]:
arima_model = auto_arima(Train, X=None, start_p=2, d=None, start_q=2, max_p=5, max_d=4, max_q=5, start_P=1, D=None, start_Q=1, max_P=2, max_D=1, max_Q=2, max_order=5, m=1, seasonal=True, stationary=False, information_criterion='aic', alpha=0.05, test='kpss', seasonal_test='ocsb', stepwise=True, n_jobs=1, start_params=None, trend=None, method='lbfgs', maxiter=50, offset_test_args=None, seasonal_test_args=None, suppress_warnings=True, error_action='trace', trace=False, random=False, random_state=None, n_fits=10, return_valid_fits=False, out_of_sample_size=0, scoring='mse', scoring_args=None, with_intercept='auto', sarimax_kwargs=None)

Print summary of auto-ARIMA model

In [29]:
arima_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                   38
Model:               SARIMAX(2, 1, 1)   Log Likelihood                -340.110
Date:                Mon, 07 Mar 2022   AIC                            690.219
Time:                        00:52:17   BIC                            698.274
Sample:                             0   HQIC                           693.059
                                 - 38                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept   -792.0638    889.779     -0.890      0.373   -2535.999     951.871
ar.L1         -0.9654      0.083    -11.594      0.000      -1.129      -0.802
ar.L2         -0.6063      0.036    -16.953      0.000      -0.676      -0.536
ma.L1          0.4863      0.216      2.252      0.024       0.063       0.910
sigma2      5.797e+06      0.176   3.29e+07      0.000     5.8e+06     5.8e+06
===================================================================================
Ljung-Box (L1) (Q):                   0.06   Jarque-Bera (JB):                 9.47
Prob(Q):                              0.81   Prob(JB):                         0.01
Heteroskedasticity (H):               0.04   Skew:                             0.73
Prob(H) (two-sided):                  0.00   Kurtosis:                         5.00
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
[2] Covariance matrix is singular or near-singular, with condition number 3.7e+23. Standard errors may be unstable.
"""

The best Auto-ARIMA model does not have the same p,d,q as the auto-ARIMA model.

Create Summary dataframe

In [30]:
Summary =pd.DataFrame(Test)

Call summary dataframe

In [31]:
Summary

,BORE_OIL_VOL
DATEPRD,
2016-04-01,3114.0
2016-05-01,3214.0
2016-06-01,3073.0
2016-07-01,1326.0


Add auto-ARIMA predictions for the next 4 months to the Summary dataframe.

In [32]:
Summary['Auto-ARIMA'] =pd.DataFrame(arima_model.predict(n_periods=4),index=Test.index)

Call summary dataframe

In [33]:
Summary

,BORE_OIL_VOL,Auto-ARIMA
DATEPRD,,
2016-04-01,3114.0,3484.921997
2016-05-01,3214.0,3066.976457
2016-06-01,3073.0,2842.155978
2016-07-01,1326.0,2520.542813


Call summary dataframe

In [34]:
Summary

,BORE_OIL_VOL,Auto-ARIMA
DATEPRD,,
2016-04-01,3114.0,3484.921997
2016-05-01,3214.0,3066.976457
2016-06-01,3073.0,2842.155978
2016-07-01,1326.0,2520.542813


 <b>4.4.2 KATS<b/> <a class="anchor" id="6"></a>

KATS which stands for Kits to Analyze Time Series is a light-weight,easy-to-use,extenable, and generalizable framework to perform time series analysis in Python. It supports forecasting, detection, feature extractionand useful utilites such as time series simulators. KATS supports the following 10 forecasting models Linear, Quadratic, ARIMA, SARIMA, Holt-Winters, Prophet, AR-Net, LSTM, Theta and VAR. 

 <b>4.4.2.1 KATS - ARIMA<b/> <a class="anchor" id="7"></a>

KATS has an ARIMA implementation and we will use this as our first model.

Call shape of dataframe

In [35]:
df1.shape

(42, 12)

Create training set by removing last 4 months of data

In [36]:
df1_train = df1.loc[:'2016-03-01'].copy()

Call shape of dataframe

In [37]:
df1_train.shape

(38, 12)

Call head of dataframe

In [38]:
df1_train.head()

,AVG_DOWNHOLE_PRESSURE,AVG_DOWNHOLE_TEMPERATURE,AVG_DP_TUBING,AVG_ANNULUS_PRESS,AVG_CHOKE_SIZE_P,AVG_WHP_P,AVG_WHT_P,DP_CHOKE_SIZE,BORE_OIL_VOL,BORE_GAS_VOL,BORE_WAT_VOL,NPD_WELL_BORE_NAME
DATEPRD,,,,,,,,,,,,
2013-02-01,242.421357,101.094214,209.964714,22.047036,96.248736,32.456643,87.408179,3.390214,25589.0,0.0,0.0,15/9-F-14
2013-03-01,245.233805,100.381593,208.418654,19.721192,61.694085,36.922413,81.552552,9.453654,18818.0,0.0,348.0,15/9-F-14
2013-04-01,244.286854,100.123739,206.884745,19.660531,60.121212,37.550047,82.163571,10.099634,8545.0,0.0,393.0,15/9-F-14
2013-05-01,238.605935,98.639355,206.072645,22.351516,97.713162,32.533387,87.742226,3.551355,27665.0,0.0,0.0,15/9-F-14
2013-06-01,244.610010,100.419686,211.495473,20.558532,85.809295,33.188373,87.409936,4.456317,22299.0,0.0,385.0,15/9-F-14


Drop 'NPD_WELL_BORE_NAME' column.

In [39]:
df1_train.drop(columns=['NPD_WELL_BORE_NAME'],inplace=True)

Create TimeSeriesData object by assigning index as time value and oil production rate as value. 

In [40]:
DF1 =TimeSeriesData(time=df1_train.index,value=df1_train['BORE_OIL_VOL'])

Create SARIMA param class

In [41]:
params = SARIMAParams(
    p = 3, 
    d=1, 
    q=0, 
    )

Initiate SARIMA model

In [42]:
m = SARIMAModel(data=DF1, params=params)

Fit SARIMA model

In [43]:
m.fit()

Generate Forecast values

In [44]:
fcst = m.predict(
    steps=4, 
    freq="MS"
    )

In [45]:
fcst

,time,fcst,fcst_lower,fcst_upper
38,2016-04-01,3890.420335,-1025.464392,8806.305062
39,2016-05-01,3707.514647,-1839.047674,9254.076968
40,2016-06-01,3708.663700,-2297.945194,9715.272593
41,2016-07-01,3794.409902,-3624.994891,11213.814695


Save Forecast values to Summary dataframe

In [46]:
Summary['KATS-ARIMAX'] =list(fcst['fcst'])

Print Forecast values

In [47]:
fcst

,time,fcst,fcst_lower,fcst_upper
38,2016-04-01,3890.420335,-1025.464392,8806.305062
39,2016-05-01,3707.514647,-1839.047674,9254.076968
40,2016-06-01,3708.663700,-2297.945194,9715.272593
41,2016-07-01,3794.409902,-3624.994891,11213.814695


Print Summary dataframe

In [48]:
Summary

,BORE_OIL_VOL,Auto-ARIMA,KATS-ARIMAX
DATEPRD,,,
2016-04-01,3114.0,3484.921997,3890.420335
2016-05-01,3214.0,3066.976457,3707.514647
2016-06-01,3073.0,2842.155978,3708.663700
2016-07-01,1326.0,2520.542813,3794.409902


<b>4.4.2.2 KATS - fbProphet<b/> <a class="anchor" id="8"></a>

Create Prophetparams class model

In [49]:
params = ProphetParams(growth='linear')

Create a prophet model instance

In [50]:
m = ProphetModel(DF1, params)

Fit fbProphet model

In [51]:
m.fit()

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Generate Forecast values

In [52]:
fcst = m.predict(
    steps=4, 
    freq="MS"
    )

Print Forecast values

In [53]:
fcst

,time,fcst,fcst_lower,fcst_upper
0,2016-04-01,-13366.501908,-15632.973261,-10987.568774
1,2016-05-01,9987.076614,7620.461603,12389.325898
2,2016-06-01,2353.400366,-51.384670,4695.909461
3,2016-07-01,1754.013981,-528.094584,4013.456154


Save Forecast values to Summary dataframe

In [54]:
Summary['KATS-fbProphet'] =list(fcst['fcst'])

Print Summary dataframe

In [55]:
Summary

,BORE_OIL_VOL,Auto-ARIMA,KATS-ARIMAX,KATS-fbProphet
DATEPRD,,,,
2016-04-01,3114.0,3484.921997,3890.420335,-13366.501908
2016-05-01,3214.0,3066.976457,3707.514647,9987.076614
2016-06-01,3073.0,2842.155978,3708.663700,2353.400366
2016-07-01,1326.0,2520.542813,3794.409902,1754.013981


Plot timeseries training set and predictions

In [56]:
m.plot()

<b>4.4.2.3 KATS - Theta model<b/> <a class="anchor" id="9"></a>

Create Thetaparams class model

In [57]:
params = ThetaParams()

Create a Theta model instance

In [58]:
m = ThetaModel(data=DF1, params=params)

Fit Theta model

In [59]:
m.fit()

C:\Users\ferna\anaconda3\envs\timeseries\lib\site-packages\kats\models\theta.py:121: FutureWarning:

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.



Generate Forecast values

In [60]:
fcst = m.predict(
    steps=4, 
    freq="MS"
    )

Print Forecast values

In [61]:
fcst

,time,fcst,fcst_lower,fcst_upper
38,2016-04-01,3560.549253,-3512.152766,10633.251273
39,2016-05-01,3293.772144,-4707.921544,11295.465832
40,2016-06-01,3026.995035,-5806.525775,11860.515845
41,2016-07-01,2760.217926,-6833.273509,12353.709361


Save Forecast values to Summary dataframe

In [62]:
Summary['KATS-Theta'] =list(fcst['fcst'])

Print Summary dataframe

In [63]:
Summary

,BORE_OIL_VOL,Auto-ARIMA,KATS-ARIMAX,KATS-fbProphet,KATS-Theta
DATEPRD,,,,,
2016-04-01,3114.0,3484.921997,3890.420335,-13366.501908,3560.549253
2016-05-01,3214.0,3066.976457,3707.514647,9987.076614,3293.772144
2016-06-01,3073.0,2842.155978,3708.663700,2353.400366,3026.995035
2016-07-01,1326.0,2520.542813,3794.409902,1754.013981,2760.217926


Plot timeseries training set and predictions

In [64]:
m.plot()

<b>4.4.2.4 KATS - Harmonic Regression model<b/> <a class="anchor" id="10"></a>

Create HarmonicRegressionModelParams class model

In [65]:
params = QuadraticModelParams()

Create a Quadratic model instance

In [66]:
m = QuadraticModel(data=DF1, params=params)

Fit Quadratic Model

In [67]:
m.fit()

Generate Forecast values

In [68]:
fcst = m.predict(
    steps=4, 
    freq="MS"
    )

Print Forecast values

In [69]:
fcst

,time,fcst,fcst_lower,fcst_upper
0,2016-04-01,2907.545756,-3687.751630,9502.843143
1,2016-05-01,2557.531677,-4192.062537,9307.125892
2,2016-06-01,2216.694606,-4710.736646,9144.125857
3,2016-07-01,1885.034541,-5244.786072,9014.855153


Save Forecast values to Summary dataframe

In [70]:
Summary['KATS-Quadratic'] =list(fcst['fcst'])

Print Summary dataframe

In [71]:
Summary

,BORE_OIL_VOL,Auto-ARIMA,KATS-ARIMAX,KATS-fbProphet,KATS-Theta,KATS-Quadratic
DATEPRD,,,,,,
2016-04-01,3114.0,3484.921997,3890.420335,-13366.501908,3560.549253,2907.545756
2016-05-01,3214.0,3066.976457,3707.514647,9987.076614,3293.772144,2557.531677
2016-06-01,3073.0,2842.155978,3708.663700,2353.400366,3026.995035,2216.694606
2016-07-01,1326.0,2520.542813,3794.409902,1754.013981,2760.217926,1885.034541


Plot timeseries training set and predictions

In [72]:
m.plot()

<b>4.4.2.5 KATS - LSTM model<b/> <a class="anchor" id="11"></a>

Create LSTMParams class model

In [73]:
params = LSTMParams(hidden_size=3,time_window=5,num_epochs = 1000)

Create an LSTM model instance

In [74]:
m = LSTMModel(data=DF1, params=params)

Fit LSTM model

In [75]:
m.fit()

Generate Forecast values

In [76]:
fcst = m.predict(
    steps=4, 
    freq="MS"
    )

Print Forecast values

In [77]:
fcst

,time,fcst,fcst_lower,fcst_upper
0,2016-04-01,-1321.122990,-1255.066840,-1387.179139
1,2016-05-01,-3541.832738,-3364.741101,-3718.924375
2,2016-06-01,-4645.572079,-4413.293475,-4877.850683
3,2016-07-01,-5221.109167,-4960.053709,-5482.164626


<b>4.4.2.6 KATS - Ensemble model<b/> <a class="anchor" id="12"></a>

The ensemble model in KATS allows us to aggregate different models in KATS. In this model we will combine the prophet, linear, quadratic and theta models and find the median of the models as a forecast.

Define the parameters for each individual forecasting model in 'EnsembleParams' class.

In [78]:
model_params = EnsembleParams(
            [
                                
                BaseModelParams("prophet", prophet.ProphetParams()),  # requires fbprophet be installed
                BaseModelParams("linear", linear_model.LinearModelParams()),
                BaseModelParams("quadratic", quadratic_model.QuadraticModelParams()),
                BaseModelParams("theta", theta.ThetaParams(m=12)),
            ]
        )

Create `KatsEnsembleParam` with detailed configurations

In [79]:
KatsEnsembleParam = {
    "models": model_params,
    "aggregation": "median",
    "seasonality_length": 0,
    "decomposition_method": "multiplicative",
    
}

Create `KatsEnsemble` model

In [80]:
m = KatsEnsemble(
    data=DF1, 
    params=KatsEnsembleParam
    )

Fit model

In [81]:
m.fit()

Generate Forecast values

In [82]:
fcst = m.predict(steps=4)

Aggregate individual model results

In [83]:
m.aggregate()

,time,fcst,fcst_lower,fcst_upper
0,2016-04-01,2311.040303,-4078.112516,8700.193123
1,2016-05-01,2925.651911,-4449.992040,10301.295862
2,2016-06-01,2285.047486,-5148.059295,8012.180314
3,2016-07-01,1819.524261,-5695.140451,7694.047188


Save Forecast values to Summary dataframe

In [84]:
Summary['KATS-Ensemble'] =list(m.aggregate()['fcst'])

Print Summary dataframe

In [85]:
Summary

,BORE_OIL_VOL,Auto-ARIMA,KATS-ARIMAX,KATS-fbProphet,KATS-Theta,KATS-Quadratic,KATS-Ensemble
DATEPRD,,,,,,,
2016-04-01,3114.0,3484.921997,3890.420335,-13366.501908,3560.549253,2907.545756,2311.040303
2016-05-01,3214.0,3066.976457,3707.514647,9987.076614,3293.772144,2557.531677,2925.651911
2016-06-01,3073.0,2842.155978,3708.663700,2353.400366,3026.995035,2216.694606,2285.047486
2016-07-01,1326.0,2520.542813,3794.409902,1754.013981,2760.217926,1885.034541,1819.524261


<b>4.4.3 SKTIME<b/> <a class="anchor" id="13"></a>

SKTIME offers scikit-learn compatible interfaces and model composition tools. It is an open-source Python toolbox for machine learning with time series. Sktime extends the scikit-learn API to time series tasks.  


<b>4.4.3.1 SKTIME-Theta model<b/> <a class="anchor" id="14"></a>

Create timeseries with the datetime index as a column.

In [86]:
df1sktime=df1.reset_index().copy()

Create training data.

In [87]:
Traindf1sktime = df1sktime.loc[:37]

Drop irrelevant columns

In [88]:
Traindf1sktime =Traindf1sktime['BORE_OIL_VOL']

Set series frequency to monthly.

In [89]:
Train.index.freq  = 'MS'

Create an Theta model instance

In [90]:
forecaster=ThetaForecaster()

Fit model

In [91]:
forecaster.fit(Traindf1sktime)

ThetaForecaster()

Generate Forecast values

In [92]:
forecaster.predict(fh=[1,2,3,4])

38    3357.509349
39    3090.732240
40    2823.955131
41    2557.178022
dtype: float64

Save Forecast values to Summary dataframe

In [93]:
Summary['SKTIME-ThetaForecaster'] = list(forecaster.predict(fh=[1,2,3,4]))

Print Summary dataframe

In [94]:
Summary

,BORE_OIL_VOL,Auto-ARIMA,KATS-ARIMAX,KATS-fbProphet,KATS-Theta,KATS-Quadratic,KATS-Ensemble,SKTIME-ThetaForecaster
DATEPRD,,,,,,,,
2016-04-01,3114.0,3484.921997,3890.420335,-13366.501908,3560.549253,2907.545756,2311.040303,3357.509349
2016-05-01,3214.0,3066.976457,3707.514647,9987.076614,3293.772144,2557.531677,2925.651911,3090.732240
2016-06-01,3073.0,2842.155978,3708.663700,2353.400366,3026.995035,2216.694606,2285.047486,2823.955131
2016-07-01,1326.0,2520.542813,3794.409902,1754.013981,2760.217926,1885.034541,1819.524261,2557.178022


<b>4.4.3.2 SKTIME-TBATS<b/> <a class="anchor" id="15"></a>

TBATS which stands for "Trigonometric seasonality, Box-Cox transformation, ARMA errors, Trend and Seasonal components" takes it root in exponential smooting methods. 

Create an TBATS model instance

In [95]:
forecaster= TBATS()

Fit TBATS model

In [96]:
forecaster.fit(Traindf1sktime)

TBATS()

Generate Forecast values

In [97]:
forecaster.predict(fh=[1,2,3,4])

38    3950.472997
39    3851.693363
40    3755.383665
41    3661.482144
Name: BORE_OIL_VOL, dtype: float64

Save Forecast values to Summary dataframe

In [98]:
Summary['SKTIME-TBATS'] = list(forecaster.predict(fh=[1,2,3,4]))

<b>4.4.3.3 SKTIME-Polynomial Trend<b/> <a class="anchor" id="16"></a>

Polynomial Trend Forecaster, forecast time series with a polynomial trend.

In [99]:
forecaster = PolynomialTrendForecaster(degree=4)

Fit Polynomial Trend model

In [100]:
forecaster.fit(Traindf1sktime)

PolynomialTrendForecaster(degree=4)

Generate Forecast values

In [101]:
forecaster.predict(fh=[1,2,3,4])

38    3298.186986
39    2743.111607
40    2090.265923
41    1322.533287
dtype: float64

Save Forecast values to Summary dataframe

In [102]:
Summary['SKTIME-PolynomialTrend'] = list(forecaster.predict(fh=[1,2,3,4]))

Print Summary dataframe

In [103]:
Summary

,BORE_OIL_VOL,Auto-ARIMA,KATS-ARIMAX,KATS-fbProphet,KATS-Theta,KATS-Quadratic,KATS-Ensemble,SKTIME-ThetaForecaster,SKTIME-TBATS,SKTIME-PolynomialTrend
DATEPRD,,,,,,,,,,
2016-04-01,3114.0,3484.921997,3890.420335,-13366.501908,3560.549253,2907.545756,2311.040303,3357.509349,3950.472997,3298.186986
2016-05-01,3214.0,3066.976457,3707.514647,9987.076614,3293.772144,2557.531677,2925.651911,3090.732240,3851.693363,2743.111607
2016-06-01,3073.0,2842.155978,3708.663700,2353.400366,3026.995035,2216.694606,2285.047486,2823.955131,3755.383665,2090.265923
2016-07-01,1326.0,2520.542813,3794.409902,1754.013981,2760.217926,1885.034541,1819.524261,2557.178022,3661.482144,1322.533287


<b>4.4.4 Silverkite <b/> <a class="anchor" id="17"></a>

The Silverkite algorithm is part of the Greykite package. Silverkite, is highly customizable, with tuning parameters to capture diverse time series characterics. The output is interpretable, allowing visualizations of the trend, seasonality, and other effects, along with their statistical significance(from: https://engineering.linkedin.com/blog/2021/greykite--a-flexible--intuitive--and-fast-forecasting-library).

Create training data for silverkite

In [104]:
TrainSilverkite = df1sktime.iloc[:38].copy()

Specify dataset information

In [105]:
 metadata = MetadataParam(
     time_col="DATEPRD",  
     value_col="BORE_OIL_VOL",  
     freq="MS" 
 )

Create a Silverkite model instance 

In [106]:
forecaster = Forecaster()

Fit Silverkite model

In [107]:
result = forecaster.run_forecast_config(
            df=TrainSilverkite,
            config=ForecastConfig(
                forecast_horizon=4,  
                coverage=0.99,  
                metadata_param=metadata
                                )
                                       )



Fitting 3 folds for each of 1 candidates, totalling 3 fits


Save forecast.

In [108]:
forecast = result.forecast

Save Forecast values to Summary dataframe

In [109]:
Summary['Silverkite'] =list(forecast.df.tail(4).round(2)['forecast'])

Print Summary dataframe

In [110]:
Summary

,BORE_OIL_VOL,Auto-ARIMA,KATS-ARIMAX,KATS-fbProphet,KATS-Theta,KATS-Quadratic,KATS-Ensemble,SKTIME-ThetaForecaster,SKTIME-TBATS,SKTIME-PolynomialTrend,Silverkite
DATEPRD,,,,,,,,,,,
2016-04-01,3114.0,3484.921997,3890.420335,-13366.501908,3560.549253,2907.545756,2311.040303,3357.509349,3950.472997,3298.186986,3359.62
2016-05-01,3214.0,3066.976457,3707.514647,9987.076614,3293.772144,2557.531677,2925.651911,3090.732240,3851.693363,2743.111607,3407.40
2016-06-01,3073.0,2842.155978,3708.663700,2353.400366,3026.995035,2216.694606,2285.047486,2823.955131,3755.383665,2090.265923,1853.69
2016-07-01,1326.0,2520.542813,3794.409902,1754.013981,2760.217926,1885.034541,1819.524261,2557.178022,3661.482144,1322.533287,1977.83


Transpose Summary dataframe

In [111]:
Summary.transpose()

DATEPRD,2016-04-01,2016-05-01,2016-06-01,2016-07-01
BORE_OIL_VOL,3114.000000,3214.000000,3073.000000,1326.000000
Auto-ARIMA,3484.921997,3066.976457,2842.155978,2520.542813
KATS-ARIMAX,3890.420335,3707.514647,3708.663700,3794.409902
KATS-fbProphet,-13366.501908,9987.076614,2353.400366,1754.013981
KATS-Theta,3560.549253,3293.772144,3026.995035,2760.217926
KATS-Quadratic,2907.545756,2557.531677,2216.694606,1885.034541
KATS-Ensemble,2311.040303,2925.651911,2285.047486,1819.524261
SKTIME-ThetaForecaster,3357.509349,3090.732240,2823.955131,2557.178022
SKTIME-TBATS,3950.472997,3851.693363,3755.383665,3661.482144
SKTIME-PolynomialTrend,3298.186986,2743.111607,2090.265923,1322.533287


Create new dataframe Summaryt which is the transposed version of Summary dataframe.

In [112]:
Summaryt=Summary.transpose()

Remove first row of Summaryt

In [113]:
Summaryt = Summaryt.iloc[1:]

Examine Summaryt dataframe.

In [114]:
Summaryt

DATEPRD,2016-04-01,2016-05-01,2016-06-01,2016-07-01
Auto-ARIMA,3484.921997,3066.976457,2842.155978,2520.542813
KATS-ARIMAX,3890.420335,3707.514647,3708.663700,3794.409902
KATS-fbProphet,-13366.501908,9987.076614,2353.400366,1754.013981
KATS-Theta,3560.549253,3293.772144,3026.995035,2760.217926
KATS-Quadratic,2907.545756,2557.531677,2216.694606,1885.034541
KATS-Ensemble,2311.040303,2925.651911,2285.047486,1819.524261
SKTIME-ThetaForecaster,3357.509349,3090.732240,2823.955131,2557.178022
SKTIME-TBATS,3950.472997,3851.693363,3755.383665,3661.482144
SKTIME-PolynomialTrend,3298.186986,2743.111607,2090.265923,1322.533287
Silverkite,3359.620000,3407.400000,1853.690000,1977.830000


Create a list of the dates out of Summary.index

In [115]:
Columns = list(Summary.index)

Add "MAPE" column to Columns list.

In [116]:
Columns.append('MAPE')

Create performance dataframe using the Columns list as the columns.

In [117]:
Performance = pd.DataFrame(columns=Columns)

Call Performance dataframe

In [118]:
Performance

,2016-04-01 00:00:00,2016-05-01 00:00:00,2016-06-01 00:00:00,2016-07-01 00:00:00,MAPE


Create list out of Summary.Columns

In [119]:
Summary_Columns = list(Summary.columns)

Remove 'BORE_OIL_VOL' from list

In [120]:
Summary_Columns.remove('BORE_OIL_VOL')

Call Summay_Columns list

In [121]:
Summary_Columns

['Auto-ARIMA',
 'KATS-ARIMAX',
 'KATS-fbProphet',
 'KATS-Theta',
 'KATS-Quadratic',
 'KATS-Ensemble',
 'SKTIME-ThetaForecaster',
 'SKTIME-TBATS',
 'SKTIME-PolynomialTrend',
 'Silverkite']

Create list out of Model names

In [122]:
Indexes = ['Auto-ARIMA',
 'KATS-ARIMAX',
 'KATS-fbProphet',
 'KATS-Theta',
 'KATS-Quadratic',
 'KATS-Ensemble',
 'SKTIME-ThetaForecaster',
 'SKTIME-TBATS',
 'SKTIME-PolynomialTrend',
 'Silverkite']

Define MAPE function

In [123]:
def mape(y, y_hat):
    return np.mean(np.abs((y - y_hat)/y)*100)

Append Performance dataframe with APE and MAPE for each model.

In [124]:
index=0
for element in range(10):
    Performance.loc[Indexes[index]] = [mape(Summary.loc['2016-04-01 00:00:00','BORE_OIL_VOL'],Summaryt.iloc[index,1]),
                                       mape(Summary.loc['2016-05-01 00:00:00','BORE_OIL_VOL'],Summaryt.iloc[index,1]),
                                       mape(Summary.loc['2016-06-01 00:00:00','BORE_OIL_VOL'],Summaryt.iloc[index,1]),
                                       mape(Summary.loc['2016-07-01 00:00:00','BORE_OIL_VOL'],Summaryt.iloc[index,1]),
                                       (mape(Summary.loc['2016-04-01 00:00:00','BORE_OIL_VOL'],Summaryt.iloc[index,1])+
                                       mape(Summary.loc['2016-05-01 00:00:00','BORE_OIL_VOL'],Summaryt.iloc[index,1])+
                                       mape(Summary.loc['2016-06-01 00:00:00','BORE_OIL_VOL'],Summaryt.iloc[index,1])+
                                       mape(Summary.loc['2016-07-01 00:00:00','BORE_OIL_VOL'],Summaryt.iloc[index,1]))/4
                                      ]
    index =index + 1

<b>4.4.5 Performance of Models <b/> <a class="anchor" id="18"></a>

Call Performance dataframe

In [125]:
Performance

,2016-04-01 00:00:00,2016-05-01 00:00:00,2016-06-01 00:00:00,2016-07-01 00:00:00,MAPE
Auto-ARIMA,1.510069,4.574472,0.196015,131.295359,34.393979
KATS-ARIMAX,19.059558,15.355154,20.648052,179.601406,58.666043
KATS-fbProphet,220.715370,210.736671,224.994358,653.173199,327.404899
KATS-Theta,5.773030,2.482021,7.184255,148.399106,40.959603
KATS-Quadratic,17.869888,20.425275,16.774107,92.875692,36.986241
KATS-Ensemble,6.048429,8.971627,4.794926,120.637399,35.113095
SKTIME-ThetaForecaster,0.747198,3.835338,0.577034,133.086896,34.561616
SKTIME-TBATS,23.689575,19.841113,25.339843,190.474613,64.836286
SKTIME-PolynomialTrend,11.910353,14.651163,10.735060,106.871162,36.041935
Silverkite,9.421965,6.017424,10.881874,156.968326,45.822397


Save Performance dataframe as CSV

In [126]:
Performance.to_csv('df1AdvancedPerformance.csv')

<b>4.5 Preprocessing and training for well that does not have typical decline curve profile <b/> <a class="anchor" id="18"></a>